# Probabilistic Ensemble Classification with TypedLogic

This tutorial demonstrates how to use TypedLogic for probabilistic ensemble classification modeling. We'll show how multiple classification methods can be combined using probabilistic reasoning, with scores propagating through an ontological hierarchy.

Our example involves classifying images into categories like animals (dog, cat) and properties (cute, fierce), where different classification methods provide confidence scores that need to be combined intelligently.

## Initial Setup

Let's start by importing the necessary modules and setting up our basic data structures.

In [1]:
from dataclasses import dataclass
from typing import Union

from typedlogic import FactMixin, axiom
from typedlogic.integrations.frameworks.owldl import Thing, TopObjectProperty
from typedlogic.extensions.probabilistic import probability

# Type aliases for clarity
ImageID = str
MethodID = str
ClassID = str
Score = float

## Classification Data Model

We'll define our core data structures for representing classification results from multiple methods.

In [2]:
@dataclass
class Image(FactMixin):
    """An image instance to be classified."""
    id: ImageID

@dataclass
class Method(FactMixin):
    """A classification method with its reliability."""
    id: MethodID
    reliability: Score  # Prior reliability of this method

@dataclass
class Classification(FactMixin):
    """A classification result from a specific method."""
    method: MethodID
    image: ImageID
    class_label: ClassID
    score: Score  # Confidence score from the method

@dataclass
class TrueClass(FactMixin):
    """The true class assignment for an image."""
    image: ImageID
    class_label: ClassID

## Ontological Hierarchy

Now we'll define our class hierarchy using OWL-DL concepts. This allows us to express relationships between classes and enable score propagation.

In [3]:
# Base classes
class Entity(Thing):
    """Root class for all entities."""
    pass

class Animal(Entity):
    """An animal entity."""
    pass

class Property(Entity):
    """A property that can be attributed to entities."""
    pass

# Specific animal classes
class Dog(Animal):
    """A dog."""
    pass

class Cat(Animal):
    """A cat."""
    pass

# Property classes
class Cute(Property):
    """The property of being cute."""
    pass

class Fierce(Property):
    """The property of being fierce."""
    pass

## Disjointness Constraints

We'll add some logical constraints to our ontology.

In [4]:
@axiom
def animal_disjointness(image: ImageID):
    """Dogs and cats are disjoint - an image cannot be both."""
    if TrueClass(image, "Dog") and TrueClass(image, "Cat"):
        assert False  # Contradiction

@axiom
def property_disjointness(image: ImageID):
    """Cute and fierce are disjoint properties."""
    if TrueClass(image, "Cute") and TrueClass(image, "Fierce"):
        assert False  # Contradiction

## Probabilistic Ensemble Rules

Now we'll define the core probabilistic rules for combining classification scores.

In [5]:
@axiom
def method_reliability_prior(method: MethodID, reliability: Score):
    """Each method has a base reliability."""
    if Method(method, reliability):
        assert probability(Classification(method, "any_image", "any_class", 1.0)) == reliability

@axiom
def score_based_classification(method: MethodID, image: ImageID, class_label: ClassID, score: Score):
    """Convert classification scores to probabilities."""
    if Classification(method, image, class_label, score) and Method(method, reliability):
        # Combine method reliability with classification score
        combined_prob = score * reliability
        assert probability(TrueClass(image, class_label)) == combined_prob

@axiom
def ensemble_aggregation(image: ImageID, class_label: ClassID):
    """Aggregate predictions from multiple methods."""
    # If multiple methods classify the same image-class pair,
    # we take the maximum probability (optimistic ensemble)
    pass  # This would be implemented with solver-specific logic

@axiom
def hierarchical_propagation(image: ImageID):
    """Propagate classifications up the hierarchy."""
    # If an image is classified as Dog, it should also be classified as Animal
    if TrueClass(image, "Dog"):
        assert TrueClass(image, "Animal")
    
    if TrueClass(image, "Cat"):
        assert TrueClass(image, "Animal")
    
    if TrueClass(image, "Cute"):
        assert TrueClass(image, "Property")
    
    if TrueClass(image, "Fierce"):
        assert TrueClass(image, "Property")

## Sample Data

Let's create some sample classification data from multiple methods.

In [6]:
# Define our classification methods with different reliabilities
methods = [
    Method("cnn_method", 0.85),      # High reliability CNN
    Method("svm_method", 0.75),      # Medium reliability SVM  
    Method("rule_method", 0.65),     # Lower reliability rule-based
]

# Sample images
images = [
    Image("image1"),
    Image("image2"), 
    Image("image3"),
    Image("image4"),
]

# Classification results from different methods
classifications = [
    # Image 1: Multiple methods agree it's a dog
    Classification("cnn_method", "image1", "Dog", 0.9),
    Classification("svm_method", "image1", "Dog", 0.8),
    Classification("rule_method", "image1", "Dog", 0.7),
    Classification("cnn_method", "image1", "Cute", 0.6),
    
    # Image 2: Methods disagree between dog and cat
    Classification("cnn_method", "image2", "Cat", 0.7),
    Classification("svm_method", "image2", "Dog", 0.6),
    Classification("rule_method", "image2", "Cat", 0.8),
    
    # Image 3: Strong agreement on cat and fierce
    Classification("cnn_method", "image3", "Cat", 0.95),
    Classification("svm_method", "image3", "Cat", 0.9),
    Classification("cnn_method", "image3", "Fierce", 0.8),
    
    # Image 4: Only property classifications
    Classification("cnn_method", "image4", "Cute", 0.7),
    Classification("rule_method", "image4", "Cute", 0.9),
]

print(f"Created {len(methods)} methods, {len(images)} images, {len(classifications)} classifications")

Created 3 methods, 4 images, 12 classifications


## Reasoning with ProbLog

Now let's use the ProbLog solver to perform probabilistic reasoning over our ensemble model.

In [7]:
from typedlogic.registry import get_solver

# Initialize ProbLog solver
solver = get_solver("problog")

# Load the current module (containing our axioms)
import sys
current_module = sys.modules[__name__]
solver.load(current_module)

# Add our facts
for method in methods:
    solver.add(method)

for image in images:
    solver.add(image)
    
for classification in classifications:
    solver.add(classification)

print("Facts loaded into solver")

TypeError: <module '__main__'> is a built-in module

## Query Results

Let's query the probabilistic model to see the ensemble predictions.

In [ ]:
# Get the probabilistic model
model = solver.model()

print("=== Ensemble Classification Probabilities ===")
print()

# Check TrueClass probabilities for each image
for image in ["image1", "image2", "image3", "image4"]:
    print(f"Image: {image}")
    
    try:
        true_class_probs = model.retrieve_probabilities('TrueClass')
        
        # Filter for this specific image
        image_probs = [(fact, prob) for fact, prob in true_class_probs 
                      if hasattr(fact, 'image') and fact.image == image]
        
        if image_probs:
            # Sort by probability descending
            image_probs.sort(key=lambda x: x[1], reverse=True)
            
            for fact, prob in image_probs:
                print(f"  {fact.class_label}: {prob:.3f}")
        else:
            print("  No classifications found")
            
    except Exception as e:
        print(f"  Error retrieving probabilities: {e}")
    
    print()

## Adding Evidence

Let's see how providing ground truth for one image affects the probabilities of others.

In [ ]:
# Add evidence that image1 is definitely a dog
solver.add_evidence(TrueClass("image1", "Dog"), True)

print("=== After adding evidence (image1 is Dog) ===")
print()

# Re-query the model
updated_model = solver.model()

for image in ["image1", "image2", "image3", "image4"]:
    print(f"Image: {image}")
    
    try:
        true_class_probs = updated_model.retrieve_probabilities('TrueClass')
        
        image_probs = [(fact, prob) for fact, prob in true_class_probs 
                      if hasattr(fact, 'image') and fact.image == image]
        
        if image_probs:
            image_probs.sort(key=lambda x: x[1], reverse=True)
            
            for fact, prob in image_probs:
                print(f"  {fact.class_label}: {prob:.3f}")
        else:
            print("  No classifications found")
            
    except Exception as e:
        print(f"  Error retrieving probabilities: {e}")
    
    print()

## Examining the Generated ProbLog Program

Let's look at the ProbLog program that was generated from our TypedLogic model.

In [ ]:
print("=== Generated ProbLog Program ===")
print()
print(solver.dump())

## Score Aggregation Strategies

Let's implement different strategies for combining scores from multiple methods.

In [ ]:
def analyze_ensemble_agreement(classifications):
    """Analyze how much the different methods agree."""
    
    print("=== Method Agreement Analysis ===")
    print()
    
    # Group by image and class
    by_image_class = {}
    for c in classifications:
        key = (c.image, c.class_label)
        if key not in by_image_class:
            by_image_class[key] = []
        by_image_class[key].append((c.method, c.score))
    
    for (image, class_label), method_scores in by_image_class.items():
        if len(method_scores) > 1:  # Multiple methods for same image-class
            scores = [score for _, score in method_scores]
            methods = [method for method, _ in method_scores]
            
            print(f"{image} -> {class_label}:")
            for method, score in method_scores:
                print(f"  {method}: {score:.2f}")
            
            # Calculate agreement metrics
            avg_score = sum(scores) / len(scores)
            score_std = (sum((s - avg_score)**2 for s in scores) / len(scores))**0.5
            
            print(f"  Average: {avg_score:.2f}, Std Dev: {score_std:.2f}")
            print()

# Run the analysis
analyze_ensemble_agreement(classifications)

## Summary

This tutorial demonstrated how to:

1. **Model multi-method classification** using TypedLogic facts and predicates
2. **Define ontological hierarchies** using OWL-DL integration for score propagation 
3. **Express disjointness constraints** to prevent contradictory classifications
4. **Combine probabilistic reasoning** with ensemble modeling using ProbLog
5. **Incorporate method reliability** as prior probabilities
6. **Add evidence** to update posterior probabilities
7. **Analyze method agreement** to understand ensemble behavior

The TypedLogic framework provides a powerful way to combine logical reasoning with probabilistic modeling, making it ideal for complex classification scenarios where multiple imperfect classifiers need to be combined intelligently.

## Next Steps

Some extensions you could explore:

- **Different aggregation strategies**: weighted voting, Bayesian model averaging
- **Learning method reliabilities**: from training data or cross-validation
- **Hierarchical classifications**: more complex ontologies with multiple inheritance
- **Temporal reasoning**: how classifications change over time
- **Active learning**: selecting which images to label next based on uncertainty